<a href="https://colab.research.google.com/github/syphax/solar-data/blob/dev/nb/Clean%20MSP%20Solar%20Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Intro

This notebook pre-processes raw downloads from https://greenmountainpower.com/account/usage and produces one cleansed file, suitable for further analysis by the `Solar Viz` notebook.

To run this script, you need access to Google Drive, and you need to copy the data from https://github.com/syphax/solar-data/tree/main/data to `/My Drive/Data/Solar` (or edit the path variable in the 2nd code block to point somewhere else).

_TODO: Load the data directly from the GitHub repo._



# Setup

In [ ]:
import os
import re

from datetime import datetime

import numpy as np
import pandas as pd
import pytz

import altair as alt
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# You can of course edit this to taste:

path = '/content/drive/MyDrive/Data/Solar/'

In [ ]:
# This will require you to click through a couple windows to
# give permission to access your GDrive.

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Load Data

This script preps data that was downloaded from [Green Mountain Power's website](https://greenmountainpower.com/account/usage/).

GMP has an excellent UI for reporting usage, and provides downloadable data in 15 minute increments (either CSV or Green Button XML). *Unfortunately* it only supports manual data downloads in 15 day (max) chunks.  
*Fortunately* it only takes a couple minutes to download several months of data. The manual downloads are just easy enough that I haven't bothered to automate the process.

Fields in the CSV downloads are:
* `ServiceAgreement`: Account info. Format is `Account Holder / Service / Service Acronym / Account Start Date / Account Status`
* `IntervalStart`: Timestamp; format is `yyyy-MM-dd-hh:mm:ss`
* `IntervalEnd`: Same, 15 minutes later. Redundant but explicit!
* `Quantity`: Amount of electricity generated
* `UnitOfMeasure` `kWh`. I love that they have an explicit UoM field!


In [ ]:
path = '/content/drive/MyDrive/Data/Solar/'

raw_input_files = os.path.join(path, 'UsageData*.csv')
joined_input_file = os.path.join(path, 'full_dataset.csv')


In [ ]:
# This should list the data files that you copied from https://github.com/syphax/solar-data/tree/main/data

!ls $path

full_dataset.csv		 UsageData_2022-02-15_14Days.csv
MA				 UsageData_2022-03-01_15Days.csv
old				 UsageData_2022-03-13_7Days.csv
UsageData_2021-05-23_14Days.csv  UsageData_2022-03-16_14Days.csv
UsageData_2021-06-06_14Days.csv  UsageData_2022-03-30_14Days.csv
UsageData_2021-06-20_14Days.csv  UsageData_2022-04-13_14Days.csv
UsageData_2021-07-04_14Days.csv  UsageData_2022-04-27_14Days.csv
UsageData_2021-07-18_14Days.csv  UsageData_2022-05-11_14Days.csv
UsageData_2021-08-01_14Days.csv  UsageData_2022-05-25_14Days.csv
UsageData_2021-08-15_14Days.csv  UsageData_2022-06-08_14Days.csv
UsageData_2021-08-29_14Days.csv  UsageData_2022-06-22_14Days.csv
UsageData_2021-09-12_14Days.csv  UsageData_2022-07-06_14Days.csv
UsageData_2021-09-26_14Days.csv  UsageData_2022-07-20_14Days.csv
UsageData_2021-10-10_14Days.csv  UsageData_2022-08-03_14Days.csv
UsageData_2021-10-24_14Days.csv  UsageData_2022-08-17_14Days.csv
UsageData_2021-11-07_12Days.csv  UsageData_2022-08-31_14Days.csv
UsageData_2021-11-18_15Day

In [ ]:
# This concatenates available data files. We will need to remove possible dupes, and check for completeness.

!cat $raw_input_files > $joined_input_file

In [ ]:
df_energy_data_raw = pd.read_csv(joined_input_file)

# Check and Clean

In [ ]:
# What fields are there?

df_energy_data_raw.dtypes

ServiceAgreement    object
IntervalStart       object
IntervalEnd         object
Quantity            object
UnitOfMeasure       object
dtype: object

In [ ]:
# Remove duplicate header rows (from the concatenation)

df_energy_data_raw = df_energy_data_raw[df_energy_data_raw['Quantity'] != 'Quantity'].copy()

In [ ]:
# Make a clean field for kWh values

df_energy_data_raw['kWh'] = 0

df_energy_data_raw['kWh'] = np.where(df_energy_data_raw['UnitOfMeasure'] == 'kWh', df_energy_data_raw['Quantity'], 0)
df_energy_data_raw['kWh'] = df_energy_data_raw['kWh'].astype(np.float64)

In [ ]:
# Quick check of values in df:

df_energy_data_raw.groupby(['ServiceAgreement', 'UnitOfMeasure'], as_index=False).agg(cnt_records=('ServiceAgreement','count'),
                                                                                      unique_dt=('IntervalStart','nunique'),
                                                                                      kwh=('kWh','sum'))

,ServiceAgreement,UnitOfMeasure,cnt_records,unique_dt,kwh
0,ACCOUNT_OWNER / Interconnected Generation Resi...,kWh,56356,55384,12172.81
1,ACCOUNT_OWNER / Residential Net Metering / N01...,kWh,56356,55384,3908.08
2,ACCOUNT_OWNER / Residential Net Metering / NGE...,kWh,56356,55384,10378.13
3,ACCOUNT_OWNER / Residential Water Heater / RE0...,kWh,56356,55384,2987.16


## Set up and correct timestamps

The timestamps in these files are in local time (discovered through inspection around the "spring forward" and "fall back" days)

To ensure that the timestamps are aligned consistently, we need to adjust for DST. Otherwise, we'll see funky inconsistencies when analyzing production by hour of day.

In the following block, we ensure that all timestamps are in EST. We choose EST because, for my main location of interest, solar noon is closer to EST noon than EDT noon (according to the [NOAA Solar Calculator](https://gml.noaa.gov/grad/solcalc/) ).

In [ ]:
# Add date field for interval starts

fmt = '%Y-%m-%d-%H:%M:%S'
df_energy_data_raw['dt_start_raw'] = pd.to_datetime(df_energy_data_raw['IntervalStart'], format=fmt, errors='coerce')
df_energy_data_raw['dt_end_raw'] = pd.to_datetime(df_energy_data_raw['IntervalEnd'], format=fmt, errors='coerce')

In [ ]:
# Convert to EST

from pytz import timezone

est = timezone('US/Eastern')

# Convert the datetime column to EST and correct for DST
# We use `ambiguous='NaT'` to clearly flag the records on the edge of DST
# We'll double-check that consumption and production are small in these times, so we can remove them
df_energy_data_raw['dt_start'] = df_energy_data_raw['dt_start_raw'].dt.tz_localize(est, ambiguous='NaT')
df_energy_data_raw['dt_end'] = df_energy_data_raw['dt_end_raw'].dt.tz_localize(est, ambiguous='NaT')


In [ ]:
df_energy_data_raw[df_energy_data_raw['dt_start'].isna()].groupby('ServiceAgreement').agg({'UnitOfMeasure':'count', 'kWh':'sum'})

,UnitOfMeasure,kWh
ServiceAgreement,,
ACCOUNT_OWNER / Interconnected Generation Residential / INTC / 06-04-2021 12:00:00AM / Active,20,0.00
ACCOUNT_OWNER / Residential Net Metering / N01 / 06-04-2021 12:00:00AM / Active,20,1.11
ACCOUNT_OWNER / Residential Net Metering / NGEN / 06-04-2021 12:00:00AM / Active,20,0.00
ACCOUNT_OWNER / Residential Water Heater / RE03 / 06-04-2021 12:00:00AM / Active,20,1.40


In [ ]:
# Assuming the above values are small (~ 1-2 kWh), drop them:

df_energy_data_raw = df_energy_data_raw[~df_energy_data_raw['dt_start'].isna()]

In [ ]:
# Check totals again:

df_energy_data_raw.groupby('ServiceAgreement').agg({'UnitOfMeasure':'count', 'kWh':'sum'})

,UnitOfMeasure,kWh
ServiceAgreement,,
ACCOUNT_OWNER / Interconnected Generation Residential / INTC / 06-04-2021 12:00:00AM / Active,56336,12172.81
ACCOUNT_OWNER / Residential Net Metering / N01 / 06-04-2021 12:00:00AM / Active,56336,3906.97
ACCOUNT_OWNER / Residential Net Metering / NGEN / 06-04-2021 12:00:00AM / Active,56336,10378.13
ACCOUNT_OWNER / Residential Water Heater / RE03 / 06-04-2021 12:00:00AM / Active,56336,2985.76


## Extract service level code

In [ ]:
# Extract service level code

p = re.compile('.*/.*/(.*)/.*/.*')

sl = df_energy_data_raw['ServiceAgreement'].str.extract(p)

df_energy_data_raw['Service'] = sl[0].str.strip()


In [ ]:
# Check for dupes

df_dupe_check = df_energy_data_raw.groupby(['Service', 'IntervalStart'], as_index=False).agg(cnt_dupes=('IntervalStart','count'))

df_dupe_check = df_dupe_check[df_dupe_check['cnt_dupes'] != 1]

df_dupe_records = df_energy_data_raw.merge(df_dupe_check, on=['Service', 'IntervalStart'], how='inner').sort_values(['IntervalStart', 'Service'])

In [ ]:
df_dupe_records.groupby('dt_start').agg({'cnt_dupes':'sum'})

,cnt_dupes
dt_start,
2021-11-07 00:00:00-04:00,16
2021-11-07 00:15:00-04:00,16
2021-11-07 00:30:00-04:00,16
2021-11-07 00:45:00-04:00,16
2021-11-18 00:00:00-05:00,16
...,...
2022-03-19 22:45:00-04:00,16
2022-03-19 23:00:00-04:00,16
2022-03-19 23:15:00-04:00,16


In [ ]:
display(df_dupe_records)

,ServiceAgreement,IntervalStart,IntervalEnd,Quantity,UnitOfMeasure,kWh,dt_start_raw,dt_end_raw,dt_start,dt_end,Service,cnt_dupes
24,ACCOUNT_OWNER / Interconnected Generation Resi...,2021-11-07-00:00:00,2021-11-07-00:15:00,0.0,kWh,0.00,2021-11-07 00:00:00,2021-11-07 00:15:00,2021-11-07 00:00:00-04:00,2021-11-07 00:15:00-04:00,INTC,2
25,ACCOUNT_OWNER / Interconnected Generation Resi...,2021-11-07-00:00:00,2021-11-07-00:15:00,0.0,kWh,0.00,2021-11-07 00:00:00,2021-11-07 00:15:00,2021-11-07 00:00:00-04:00,2021-11-07 00:15:00-04:00,INTC,2
8,ACCOUNT_OWNER / Residential Net Metering / N01...,2021-11-07-00:00:00,2021-11-07-00:15:00,0.06,kWh,0.06,2021-11-07 00:00:00,2021-11-07 00:15:00,2021-11-07 00:00:00-04:00,2021-11-07 00:15:00-04:00,N01,2
9,ACCOUNT_OWNER / Residential Net Metering / N01...,2021-11-07-00:00:00,2021-11-07-00:15:00,0.06,kWh,0.06,2021-11-07 00:00:00,2021-11-07 00:15:00,2021-11-07 00:00:00-04:00,2021-11-07 00:15:00-04:00,N01,2
0,ACCOUNT_OWNER / Residential Net Metering / NGE...,2021-11-07-00:00:00,2021-11-07-00:15:00,0.0,kWh,0.00,2021-11-07 00:00:00,2021-11-07 00:15:00,2021-11-07 00:00:00-04:00,2021-11-07 00:15:00-04:00,NGEN,2
...,...,...,...,...,...,...,...,...,...,...,...,...
6143,ACCOUNT_OWNER / Residential Net Metering / N01...,2022-03-19-23:45:00,2022-03-20-00:00:00,0.07,kWh,0.07,2022-03-19 23:45:00,2022-03-20 00:00:00,2022-03-19 23:45:00-04:00,2022-03-20 00:00:00-04:00,N01,2
5374,ACCOUNT_OWNER / Residential Net Metering / NGE...,2022-03-19-23:45:00,2022-03-20-00:00:00,0.0,kWh,0.00,2022-03-19 23:45:00,2022-03-20 00:00:00,2022-03-19 23:45:00-04:00,2022-03-20 00:00:00-04:00,NGEN,2
5375,ACCOUNT_OWNER / Residential Net Metering / NGE...,2022-03-19-23:45:00,2022-03-20-00:00:00,0.0,kWh,0.00,2022-03-19 23:45:00,2022-03-20 00:00:00,2022-03-19 23:45:00-04:00,2022-03-20 00:00:00-04:00,NGEN,2
6910,ACCOUNT_OWNER / Residential Water Heater / RE0...,2022-03-19-23:45:00,2022-03-20-00:00:00,0.0,kWh,0.00,2022-03-19 23:45:00,2022-03-20 00:00:00,2022-03-19 23:45:00-04:00,2022-03-20 00:00:00-04:00,RE03,2


As we've already dealt with the DST issue, we can safely drop duplicate records, which are likely due to overlapping extracts.

In [ ]:
sh0 = df_energy_data_raw.shape

df_solar_data = df_energy_data_raw.copy()

sh0 = df_solar_data.shape

df_solar_data = df_solar_data.drop_duplicates()

sh1 = df_solar_data.shape

cnt_dupes = sh0[0] - sh1[0]

# Number of hours, as data is every 15 minutes:
cnt_unique_services = df_solar_data['Service'].nunique()

cnt_hours = sh1[0] / 4.0 / cnt_unique_services
cnt_years = cnt_hours / (365.25 * 24)

print("{:,} original entries. Removed {:,} duplicate entries; {:,} left.".format(sh0[0], cnt_dupes, sh1[0]))
print("{:,} hours of data ({:.2f} years).".format(cnt_hours, cnt_years))

225,344 original entries. Removed 3,840 duplicate entries; 221,504 left.
13,844.0 hours of data (1.58 years).


In [ ]:
# Drop ServiceAgreement (which contains some PID):

df_solar_data = df_solar_data.drop('ServiceAgreement', axis=1)

In [ ]:
df_solar_data.sample(8)

,IntervalStart,IntervalEnd,Quantity,UnitOfMeasure,kWh,dt_start_raw,dt_end_raw,dt_start,dt_end,Service
144682,2022-05-23-19:45:00,2022-05-23-20:00:00,0.01,kWh,0.01,2022-05-23 19:45:00,2022-05-23 20:00:00,2022-05-23 19:45:00-04:00,2022-05-23 20:00:00-04:00,INTC
10841,2021-06-20-21:45:00,2021-06-20-22:00:00,0.0,kWh,0.00,2021-06-20 21:45:00,2021-06-20 22:00:00,2021-06-20 21:45:00-04:00,2021-06-20 22:00:00-04:00,NGEN
203751,2022-10-24-00:15:00,2022-10-24-00:30:00,0.0,kWh,0.00,2022-10-24 00:15:00,2022-10-24 00:30:00,2022-10-24 00:15:00-04:00,2022-10-24 00:30:00-04:00,INTC
88569,2022-01-08-02:15:00,2022-01-08-02:30:00,0.24,kWh,0.24,2022-01-08 02:15:00,2022-01-08 02:30:00,2022-01-08 02:15:00-05:00,2022-01-08 02:30:00-05:00,N01
132587,2022-04-25-20:30:00,2022-04-25-20:45:00,0.0,kWh,0.00,2022-04-25 20:30:00,2022-04-25 20:45:00,2022-04-25 20:30:00-04:00,2022-04-25 20:45:00-04:00,RE03
132575,2022-04-25-17:30:00,2022-04-25-17:45:00,0.0,kWh,0.00,2022-04-25 17:30:00,2022-04-25 17:45:00,2022-04-25 17:30:00-04:00,2022-04-25 17:45:00-04:00,RE03
47182,2021-09-13-09:30:00,2021-09-13-09:45:00,0.85,kWh,0.85,2021-09-13 09:30:00,2021-09-13 09:45:00,2021-09-13 09:30:00-04:00,2021-09-13 09:45:00-04:00,INTC
219053,2022-11-26-05:00:00,2022-11-26-05:15:00,0.0,kWh,0.00,2022-11-26 05:00:00,2022-11-26 05:15:00,2022-11-26 05:00:00-05:00,2022-11-26 05:15:00-05:00,INTC


# Save Clean Dataset

In [ ]:
df_solar_data.to_csv(os.path.join(path, 'full_dataset.csv'), index=False)